In [4]:
import datasets
import numpy as np
from transformers import BertTokenizerFast
from transformers import AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification


In [3]:
conll2003=datasets.load_dataset("conll2003")

Dataset conll2003 downloaded and prepared to /Users/roshinjohnson/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 426.74it/s]


In [9]:
conll2003["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [11]:
conll2003

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [12]:
conll2003.shape

{'train': (14041, 5), 'validation': (3250, 5), 'test': (3453, 5)}

In [22]:
conll2003["train"].features['chunk_tags']

Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None)

In [16]:
tokenizer= BertTokenizerFast.from_pretrained("bert-base-uncased")

In [17]:
type(conll2003["train"].features['ner_tags'])

datasets.features.features.Sequence

In [25]:
example=conll2003["train"][0]

tokenized_input=tokenizer(example["tokens"], is_split_into_words=True)

In [26]:
tokenized_input

{'input_ids': [101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [27]:
tokens=tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

In [28]:
tokens

['[CLS]',
 'eu',
 'rejects',
 'german',
 'call',
 'to',
 'boycott',
 'british',
 'lamb',
 '.',
 '[SEP]']

In [29]:
len(tokenized_input["input_ids"]), len(example["ner_tags"])

(11, 9)

In [30]:
word_ids= tokenized_input.word_ids()

In [31]:
word_ids

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]

In [34]:
for i in word_ids:
    if(i==None):
        print(22)

22
22


In [33]:
type(tokenized_input)

transformers.tokenization_utils_base.BatchEncoding

In [57]:
def tokenize_and_align_labels(examples,label_all_tokens=True):
    tokenized_input=tokenizer(examples["tokens"], truncation=True,  is_split_into_words=True)
    word_ids= tokenized_input.word_ids()
    label=examples["ner_tags"]
    label_new=[]
    #print(label)
    previous_token=None
    for i in word_ids:
        #print(i)
        if(i==None):
            label_new.append(-100)
        elif(label[i] !=previous_token):
            previous_token=label[i]
            label_new.append(label[i] )
        else:
            #print(i)
            label_new.append(label[i] if label_all_tokens else -100)
    tokenized_input["labels"]=label_new
    return tokenized_input

        
    

In [63]:
def tokenize_and_align_labels(examples,label_all_tokens=True):
    tokenized_input=tokenizer(examples["tokens"], truncation=True,  is_split_into_words=True)
    l=[]
    for i, label in enumerate(examples["ner_tags"]):
        
        word_ids= tokenized_input.word_ids(batch_index=i)

        label_new=[]
        #print(label)
        previous_token=None
        for i in word_ids:
            #print(i)
            if(i==None):
                label_new.append(-100)
            elif(label[i] !=previous_token):
                previous_token=label[i]
                label_new.append(label[i] )
            else:
                #print(i)
                label_new.append(label[i] if label_all_tokens else -100)
        l.append(label_new)
    tokenized_input["labels"]=l
    return tokenized_input

In [64]:
example["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [67]:
tokenize_and_align_labels(conll2003["train"][4:7])

{'input_ids': [[101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102], [101, 1000, 2057, 2079, 1050, 1005, 1056, 2490, 2151, 2107, 12832, 2138, 2057, 2079, 1050, 1005, 1056, 2156, 2151, 5286, 2005, 2009, 1010, 1000, 1996, 3222, 1005, 1055, 2708, 14056, 24794, 2271, 3158, 4315, 14674, 2409, 1037, 2739, 27918, 1012, 102], [101, 2002, 2056, 2582, 4045, 2817, 2001, 3223, 1998, 2065, 2009, 2001, 2179, 2008, 2895, 2001, 2734, 2009, 2323, 2022, 2579, 2011, 1996, 2647, 2586, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [62]:
x=tokenize_and_align_labels(conll2003["train"][4])
len(x["input_ids"]), len(x["labels"])

(39, 39)

In [68]:
trained_dataset=conll2003.map(tokenize_and_align_labels, batched=True)

In [69]:
model=AutoModelForTokenClassification(from_pretrained="bert-base-uncased")

ImportError: 
AutoModelForTokenClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
